In [1]:
# Import all modules.
import numpy as np
import pandas as pd
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [2]:
# Working on zip file.
# My idea is to create folder from zip file as to easy in use.

import zipfile

zip_file= "/content/New assi.zip"

#  extraction directory
extract_dir = "my_images"

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [3]:
# Similar for Masks.
import zipfile

# Specify the name of the uploaded zip file
zip_file= "/content/Q2.zip"

# Specify the extraction directory
extract_dir = "my_zip_images"

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [4]:
# Set the path to your images folder
image_folder = "/content/my_images/images"
mask_folder= "/content/my_zip_images/masks"

In [5]:
from tqdm import tqdm

In [6]:
# Load and preprocess image.
def preprocess_data(image_folder,mask_folder):
  image=[]
  mask_=[]

  image_name=os.listdir(image_folder)
  mask_name=os.listdir(mask_folder)   # list of filenames

  mask_name.sort()
  image_name.sort()  # To match image and mask name

  for image_name,mask_name in tqdm(zip(image_name,mask_name),total=len(image_name)):

    image_p=os.path.join(image_folder,image_name)
    img=cv2.imread(image_p)
    img=cv2.resize(img,(800,540))

    img=img.astype('float32')/255.0

    # Similarly for mask image.
    mask_p=os.path.join(mask_folder,mask_name)
    mask=cv2.imread(mask_p,cv2.IMREAD_GRAYSCALE)
    mask=cv2.resize(mask,(800,540))

    mask=mask.astype('float32')/255.0
    mask=np.where(mask>0,1,0)

    mask_.append(mask)
    image.append(img)
  return np.array(image), np.array(mask_)





In [7]:
# Now it's time to load data.
imgs,masks=preprocess_data(image_folder,mask_folder)

100%|██████████| 317/317 [00:03<00:00, 82.77it/s]


In [8]:
imgs.shape

(317, 540, 800, 3)

In [9]:
masks.shape

(317, 540, 800)

In [10]:
# Now its time for splitting.
X_train, X_test, y_train, y_test = train_test_split(imgs, masks, test_size=0.3, random_state=42)

In [11]:
# To check nonzero pixcels in image.
np.nonzero(X_train[2])

(array([  1,   1,   1, ..., 538, 538, 538]),
 array([ 99,  99,  99, ..., 784, 784, 784]),
 array([0, 1, 2, ..., 0, 1, 2]))

In [12]:
# Now time for model construction.
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, concatenate,UpSampling2D,MaxPooling2D


In [13]:
# I tried U-Net model especially for segmentation.
# here I refer model Artitecture from Kaggle,Chatgpt.

def create_unet_model(inp_shape):
    inputs = Input(inp_shape)
    # Here i need to define Encoder Artitecture and for decoder Artit

    # Encoder
    conv1 = Conv2D(62, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D()(conv1)
    conv2 = Conv2D(126, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D()(conv2)

    # Decoder
    up1 = UpSampling2D()(pool2)
    concat1 = concatenate([conv2, up1], axis=-1)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(concat1)
    up2 = UpSampling2D()(conv3)
    concat2 = concatenate([conv1, up2], axis=-1)
    segmentation_output = Conv2D(1, 1, activation='sigmoid')(concat2)

    model = Model(inputs, segmentation_output)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model

NOTE: My computational power + collab gpu memory exceeds so i didn't train for more epochs

In [14]:
import tensorflow as tf

# Limit GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [15]:
# Enable mixed precision training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [16]:
# Its time for Training.
input_shape = X_train[0].shape
seg_model = create_unet_model(input_shape)
seg_model.fit(X_train, y_train, epochs=1, batch_size=8, validation_data=(X_test, y_test))

28/28 [==============================] - 438s 6s/step - loss: 0.2277 - accuracy: 0.9619 - val_loss: 0.0946 - val_accuracy: 0.9938


In [18]:
# Save the model weights
weights_path = '/content/my_images.h5'
seg_model.save_weights(weights_path)